# モンテカルロ法 三目並べ

In [2]:
def state_encode(state3):
    convert = [[0,1,2,3,4,5,6,7,8],# 元の状態
              [2,1,0,5,4,3,8,7,6], # 変換(2)
              [6,3,0,7,4,1,8,5,2], # 変換(3)
              [0,3,8,1,4,7,2,5,8], # 変換(4)
              [8,7,6,5,4,3,2,1,0], # 変換(5)
              [6,7,8,3,4,5,0,1,2], # 変換(6)
              [2,5,8,1,4,7,0,3,6], # 変換(7)
              [8,5,2,7,4,1,6,3,0]  # 変換(8)
              ]
    power = np.array([3**i for i in range(8,-1,-1)], dtype=float)
    
    cands = []
    for conv in convert:
        # 並び替え
        state = [state3[i] for i in conv]
        
        cands += [sum(state*power)]
    # 8個の候補のうち一番小さいものを選ぶ
    return min(cands)+1

In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt

def action_decision(t, state3, policy):
    action = None
    if t == 0:
        action = 0
    else:
        while(True):
            sum_pol = 0
            ran = random.random()
            for i in range(len(state3)):
                sum_pol += policy[i]
                if sum_pol > ran:
                    break
                
            if state3[i] == 0:
                action = i
                break
            
    return action

def opponent_action(state3):
    lines = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
    for line in lines:
        state_line = [state3[i] for i in line]
        count0 = state_line.count(0)
        count1 = state_line.count(1)
        # リーチされたら負けを防ぐ
        if count0 == 1 and count1 == 2:
            return line[state_line.index(0)]
            
    while(True):
        ran = random.randint(0, 8)
        if state3[ran] == 0:
            return ran
    

def check_fin(state3):
    fin = 0
    lines = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
    for line in lines:
        state_line = [state3[i] for i in line]
        count1 = state_line.count(1)
        if count1 == 3:
            return 1 # 勝ち
        count2 = state_line.count(2)
        if count2 == 3:
            return 2 # 負け
    if 0 not in state3:
        return 3 # 引き分け
    return 0 # 継続

def reward_decision(fin):
    if fin == 1:
        return 10
    elif fin == 2:
        return -10
    elif fin == 3:
        return 0
    return None

def action_train(policy, t, state3):
    count1, count2 = 0, 0
    for i in range(len(state3)):
        if state3[i] == 1:
            count1 += 1
        elif state3[i] == 2:
            count2 += 1
            
    if count1 == count2:
        action = action_decision(t, state3, policy)
        state3[action] = 1
        fin = check_fin(state3)
        reward = reward_decision(fin)
        return action, reward, state3, fin
    else:
        action = opponent_action(state3)
        state3[action] = 2
        fin = check_fin(state3)
        reward = reward_decision(fin)
        return action, reward, state3, fin


# L:価値反復数（価値関数の更新回数）, M:エピソード数
def MonteCarloPolycyIteration(L, M, options):
    nstates = 3**9   # 状態数
    nactions = 9     # 行動数
    T = 9            # 最大ステップ数
    # Q関数の初期化
    Q = np.zeros([nstates, nactions])
    rate = []
    
    for l in range(L):
        states = np.ones([M, T])
        actions = np.ones([M, T])
        rewards = np.ones([M, T])
        drewards = np.ones([M, T])
        visits = np.ones([nstates, nactions])
        results = np.zeros([M])
        np.random.seed(0)
        
        # エピソード
        for m in range(M):
            state3 = np.zeros([nactions])
            
            # ステップ
            for t in range(T):
                # 状態のエンコード
                state = state_encode(state3)
                # 政策の生成
                policy = np.zeros([nactions])
                
                if options['pmode'] == 1:    # greedy
                    v = max(Q[state])
                    a = np.where(Q[state]==v)[0][0]
                    policy[a] = 1
                    
                elif options['pmode'] == 2:  # e-greedy
                    v = max(Q[state])
                    a = np.where(Q[state]==v)[0][0]
                    policy = np.ones([nactions]) * options['epsilon'] / nactions
                    policy[a] = 1 - options['epsilon'] + options['epsilon'] / nactions
                    
                elif options['pmode'] == 3:  # softmax
                    policy = np.exp(Q[state] / options['tau']) / sum(np.exp(Q[state] / options['tau']))
                
                # 行動選択および実行
                action, reward, state3, fin = action_train(policy, t, state3)
                # 状態、行動、報酬、出現回数の更新
                states[m][t] = state
                actions[m][t] = action
                rewards[m][t] = reward
                visits[state][action] += 1
                
                # ゲーム終了
                if fin > 0:
                    results[m] = fin
                    # 割引き報酬和の計算
                    drewards[m][t] = rewards[m][t]
                    for pstep in range(t-1,-1,-1):
                        drewards[m][pstep] = options['gamma'] * drewards[m][pstep+1]
                    break    
        
        # 状態行動価値関数の計算
        Q = np.zeros([nstates, nactions])
                
        for m in range(M):
            for t in range(T):
                s = states[m][t]
                if s == 0:
                    break
                a = actions[m][t]
                Q[s][a] += drewards[m][t]
           
        Q = Q / visits
        
        rate.append(len(results[results==1]) / M)
        
        print("Win: {0}/{1}, Draw: {2}/{3}, Lose: {4}/{5}".format(len(results[results==1]),M,
                                                                  len(results[results==3]),M,
                                                                  len(results[results==2]),M))
        
    return rate

In [4]:
if __name__=='__main__':
    # オプション　pmode:(greedy, e-greedy, softmax), epsilon:ランダム行動確率, gamma:割引率
    options = {'pmode':2,'epsilon':0.1,'gamma':0.9}
    rate = MonteCarloPolycyIteration(100,100,options) # 試行回数,エピソード数,オプション
    
    plt.clf()
    plt.plot(range(len(rate)), rate)
    plt.savefig('montecarlo.png')

/Users/daisuke/.pyenv/versions/anaconda3-2.4.0/lib/python3.5/site-packages/ipykernel/__main__.py:121: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/daisuke/.pyenv/versions/anaconda3-2.4.0/lib/python3.5/site-packages/ipykernel/__main__.py:122: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/daisuke/.pyenv/versions/anaconda3-2.4.0/lib/python3.5/site-packages/ipykernel/__main__.py:135: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/daisuke/.pyenv/versions/anaconda3-2.4.0/lib/python3.5/site-packages/ipykernel/__main__.py:155: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Win: 17/100, Draw: 55/100, Lose: 28/100
Win: 36/100, Draw: 41/100, Lose: 23/100
Win: 49/100, Draw: 33/100, Lose: 18/100
Win: 59/100, Draw: 22/100, Lose: 19/100
Win: 72/100, Draw: 18/100, Lose: 10/100
Win: 61/100, Draw: 19/100, Lose: 20/100
Win: 62/100, Draw: 25/100, Lose: 13/100
Win: 68/100, Draw: 21/100, Lose: 11/100
Win: 66/100, Draw: 23/100, Lose: 11/100
Win: 65/100, Draw: 23/100, Lose: 12/100
Win: 59/100, Draw: 29/100, Lose: 12/100
Win: 70/100, Draw: 25/100, Lose: 5/100
Win: 70/100, Draw: 23/100, Lose: 7/100
Win: 72/100, Draw: 16/100, Lose: 12/100
Win: 79/100, Draw: 8/100, Lose: 13/100
Win: 76/100, Draw: 16/100, Lose: 8/100
Win: 67/100, Draw: 21/100, Lose: 12/100
Win: 73/100, Draw: 23/100, Lose: 4/100
Win: 65/100, Draw: 23/100, Lose: 12/100
Win: 80/100, Draw: 14/100, Lose: 6/100
Win: 60/100, Draw: 25/100, Lose: 15/100
Win: 70/100, Draw: 24/100, Lose: 6/100
Win: 76/100, Draw: 22/100, Lose: 2/100
Win: 75/100, Draw: 24/100, Lose: 1/100
Win: 69/100, Draw: 24/100, Lose: 7/100
Win: 78/10